In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [7]:
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path
import os 


In [16]:
def perform():
    face_classifier = cv2.CascadeClassifier('haarcasacde_facefrontal_default.xml')
    BASE_DIR = r'C:\Users\parab\Desktop\python\flask\env\src\RPM'
    def face_extraction(img):
        faces = face_classifier.detectMultiScale(img,1.3,5)
        
        if faces is ():
            return None
        
        for (x,y,w,h) in faces :
            x=x-10
            y=y-10
            cropped_face = img[y:y+h+50 , x:x+w+50]
            
        return cropped_face

    cap = cv2.VideoCapture(0)
    count =0 

    while (cap.isOpened()):
        success , img = cap.read()
        if face_extraction(img) is not None :
            count = count+1
            face = cv2.resize(face_extraction(img),(224,224))
            
            file_name= str(count)+'.jpg'
            if count < 40 :
                file_name_path = os.path.join(BASE_DIR,'faces','train','person',file_name)
            #'C:/Users/parab/Desktop/python/opencv/images/face_recog_train/aryan/'
            else:
                file_name_path = os.path.join(BASE_DIR,'faces','test','person',file_name)
            cv2.imwrite(file_name_path,face)
            
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Face Cropper',face)
            
        else:
            print("Face not Found")
            pass
        
        
        if cv2.waitKey(1) == 1000 or count == 60:
            break

    cap.release()
    cv2.destroyAllWindows()




<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-16-8b9411a072b5>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [58]:


def make_model():
  IMAGE_SIZE = [224,224]

  #train_path = 'images/mask_nomask/'
  #valid_path = 'images/mask_nomask_test/'

  vgg = VGG16(input_shape=IMAGE_SIZE + [3],weights ='imagenet',include_top = False)

  for layer in vgg.layers:
      layer.trainable = False
          
  folders = glob('faces/train/*')

  x = Flatten()(vgg.output)

  prediction = Dense(1,activation = 'sigmoid')(x)



  model= Model(inputs=vgg.input , outputs = prediction)

  model.summary()

  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy']
                )

  

  train_datagen = ImageDataGenerator(rescale = 1./255,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)

  test_datagen = ImageDataGenerator(rescale = 1./255)

  training_set = train_datagen.flow_from_directory('faces/train',
                                                   target_size = (224,224),
                                                   batch_size = 32,
                                                   class_mode = 'binary')

  test_set = test_datagen.flow_from_directory('faces/test',
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = 'binary')

  '''r=model.fit_generator(training_set,
                           samples_per_epoch = 8000,
                           nb_epoch = 5,
                           validation_data = test_set,
                           nb_val_samples = 2000)'''

  # fit the model

  r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=3,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
  )
  
  
  model.save('sface_recog.h5')






In [56]:
perform()
make_model()

Face not Found
Face not Found
Face not Found
Face not Found
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

ValueError: in user code:

    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\parab\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

In [34]:

face_classifier = cv2.CascadeClassifier('haarcasacde_facefrontal_default.xml')
def face_extraction(img):
    faces = face_classifier.detectMultiScale(img,1.3,5)
        
    if faces is ():
        print("no face")
        return None
        
    for (x,y,w,h) in faces :
        x=x-10
        y=y-10
        cropped_face = img[y:y+h+50 , x:x+w+50]
            
    return cropped_face

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-34-f1f0fe866601>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [43]:
model = load_model('sface_recog.h5')


In [69]:
img = cv2.imread('images/passport1.jpeg')
img = face_extraction(img)
cv2.imwrite('images\made_img.jpg',img)

True

In [72]:
from tensorflow.keras.models import load_model


img = image.load_img('images/made_img.jpg', target_size=(224, 224))


x = image.img_to_array(img)
x = np.true_divide(x, 255)
    ## Scaling
#x=x/255
x = np.expand_dims(x, axis=0)
print(x.shape)
pred = model.predict(x)[0][0]
print(pred)

(1, 224, 224, 3)
0.28162432
